In [1]:
include("milkbot.jl")
using  .Milkbot, CSV, HTTP, StructTypes,  Plots, DataFrames, JSON3, TensorCast, Setfield

apikey = "<your key here>"  #enter your API key here
file = "supplementalDataS1/Additional file 9.csv" #data file (use pwd() to see your current working directory)

#server = "https://milkbot.com/"
#server = "http://localhost:8080"
server = "https://europe-west1-numeric-analogy-337601.cloudfunctions.net/milkBot-fitter"
cred = Credentials(server, apikey)

const df = CSV.read(file,DataFrame)

#define a structure to match the input file
struct RowType1
    lactkey::Integer
    breed::String
    parity::Integer
    scale::Number
    ramp::Number 
    decay::Number
    offset::Number
    n::Integer
    se::Number
    dim::Vector
    milk::Vector
end

#function to map dataframe fields to the RowType
#we need to lift (un-flatten) the DIM,milk values 
RowType1(d:: DataFrame, r::Integer ) = RowType1(
    d[r,:lactkey], 
    d[r,:Breed],      
    d[r,:Parity],
    d[r,:Scale],
    d[r,:Ramp],
    d[r,:Decay],
    d[r,:Offset],
    d[r,:points],
    d[r,:RMSE],
    filter(!isnothing, map(n -> d[r, "dim$(n)"], 1:9)),
    filter(!isnothing, map(n ->  d[r, "milk$(n)"], 1:9)) 
)
#below used by JSON3
StructTypes.StructType(::Type{Priors}) = StructTypes.Struct()
StructTypes.StructType(::Type{MBParams}) = StructTypes.Struct()
StructTypes.StructType(::Type{Fit}) = StructTypes.Struct()
StructTypes.StructType(::Type{FittingJob}) = StructTypes.Struct()
StructTypes.StructType(::Type{LactationSet}) = StructTypes.Struct()

pr = getPriorsTree(cred) #then cut out Hostein/Parity2/priors below 
jsp = JSON3.read(pr).content.node.priorsTree[1].content.node.priorsTree[2].content.node.priors 
h2priors = JSON3.read(JSON3.write(jsp), Priors) #no PriorsTree struct

lacFromRow(r::RowType1) = begin
    pts = map(i -> MilkPoint(r.dim[i], r.milk[i]) , 1:(min(length(r.dim),length(r.milk))))
    l = Lactation(string(r.lactkey),r.breed,r.parity, pts)
    f = Fit(string(r.lactkey),  MBParams(r.scale,r.ramp,r.decay,r.offset), r.n, r.se, nothing) #no path yet
FittedLactation(l,f)
end

r = RowType1(df,42)
l = lacFromRow(r).lac


time50SingleFitsWithRead() = begin
    @elapsed for i in 1:50
        f = fitLactation(lacFromRow(RowType1(df,i)).lac, h2priors, cred,false)
        #display(f) #testing only, then comment out 21.616168787
    end
end

time50SingleFits(n:: Integer) = begin
    r = RowType1(df,1)
    l = lacFromRow(r).lac
    @elapsed (for i in 1:n
       fitLactation(lacFromRow(RowType1(df,i)).lac, h2priors, cred,false)
    end)
end

timeSetOf50() = begin
    ls = map(r -> lacFromRow(RowType1(df,r)).lac, 1:nrow(df))
    println("$(length(ls)) lactations in set")
    set = LactationSet("test set", ls,"Kg")
    fj = FittingJob(set, pr,nothing)
    @elapsed fitLactationSet(fj, cred)
end

timeLoadSetOf50() = begin
    @elapsed begin 
        ls = map(r -> lacFromRow(RowType1(df,r)).lac, 1:nrow(df))
        set = LactationSet("test set", ls,"Kg")
        fj = FittingJob(set, pr,nothing)
    end
end

t50With = time50SingleFitsWithRead()
t50 = time50SingleFitsWithRead()
l50 = timeLoadSetOf50()
fit50 = timeSetOf50()

println("Read 50 lactations from dataframe and fit individually in $(t50With) sec ->  $(t50With*20/60) minutes per thousand")
println("Fit a lactation with 9 points 50 times in $(t50) sec ->  $(t50*20/60) minutes per thousand")
println("Load a fitting job of 50  $(l50) sec ->  $(l50*20/60) minutes per thousand")
println("Fit a fitting job of 50  $(fit50) sec ->  $(fit50*20/60) minutes per thousand")




50 lactations in set
Read 50 lactations from dataframe and fit individually in 11.957889375 sec ->  3.985963125 minutes per thousand
Fit a lactation with 9 points 50 times in 8.824913705 sec ->  2.9416379016666667 minutes per thousand
Load a fitting job of 50  0.000872759 sec ->  0.0002909196666666667 minutes per thousand
Fit a fitting job of 50  0.825484019 sec ->  0.27516133966666667 minutes per thousand


┌ Warning: thread = 1 warning: parsed expected 33 columns, but didn't reach end of line around data row: 9. Parsing extra columns and widening final columnset
└ @ CSV /Users/jimehrlich/.julia/packages/CSV/cHirL/src/file.jl:579
┌ Warning: thread = 1 warning: only found 31 / 35 columns around data row: 10. Filling remaining columns with `missing`
└ @ CSV /Users/jimehrlich/.julia/packages/CSV/cHirL/src/file.jl:578
┌ Warning: thread = 1 warning: only found 31 / 35 columns around data row: 12. Filling remaining columns with `missing`
└ @ CSV /Users/jimehrlich/.julia/packages/CSV/cHirL/src/file.jl:578
┌ Warning: thread = 1 warning: only found 33 / 35 columns around data row: 13. Filling remaining columns with `missing`
└ @ CSV /Users/jimehrlich/.julia/packages/CSV/cHirL/src/file.jl:578
┌ Warning: thread = 1 warning: only found 33 / 35 columns around data row: 14. Filling remaining columns with `missing`
└ @ CSV /Users/jimehrlich/.julia/packages/CSV/cHirL/src/file.jl:578
┌ Warning: thread = 1

In [2]:
ls = map(r -> lacFromRow(RowType1(df,r)).lac, 1:nrow(df))
println("$(length(ls)) lactations in set")
set = LactationSet("test set", ls,"Kg")
fj = FittingJob(set, priors,nothing)

UndefVarError: UndefVarError: priors not defined